In [3]:
import numpy as np
import pandas as pd
import warnings

from sklearn.metrics import root_mean_squared_error , r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor



<frozen importlib._bootstrap>:488: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [4]:
df = pd.read_csv("data/student_data.csv")

In [5]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
X = df.drop(columns='math score',axis=1)
Y = df['math score']

In [7]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features)
    ]
)

In [8]:
X = preprocessor.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=7)

X_train.shape , X_test.shape

((800, 19), (200, 19))

In [10]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    rmse = root_mean_squared_error(true,predicted)
    r2_square = r2_score(true,predicted)
    return mae , rmse , r2_square

In [11]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'K-NeighborsRegressor': KNeighborsRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'XGBRegressor': XGBRegressor(),
    'CatBoostRegressor': CatBoostRegressor(verbose=False),
    'AdaBoostRegressor': AdaBoostRegressor()
}

model_list = []
r2_list = []

In [13]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,Y_train)

    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(Y_train,Y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(Y_test,Y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:4f}".format(model_train_rmse))
    print("- Mean Absolute error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print("-------------------------------------------------")

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:4f}".format(model_test_rmse))
    print("- Mean Absolute error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')



LinearRegression
Model performance for Training set
- Root Mean Squared Error: 5.353941
- Mean Absolute error: 4.2625
- R2 Score: 0.8788
-------------------------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.548254
- Mean Absolute error: 4.4925
- R2 Score: 0.8454


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.523650
- Mean Absolute error: 5.1625
- R2 Score: 0.8201
-------------------------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.754414
- Mean Absolute error: 5.2640
- R2 Score: 0.7708


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.292758
- Mean Absolute error: 4.2070
- R2 Score: 0.8816
-------------------------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.478869
- Mean Absolute error: 4.4190
- R2 Score: 0.8492


K-NeighborsRegressor
Model performance for Training set
- Root Mean Squared Error: 5.894357
- Mean Absolute

In [14]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name','R2_Score']).sort_values(by=['R2_Score'],ascending=False)

,Model Name,R2_Score
11,Ridge,0.849202
2,Ridge,0.849202
0,LinearRegression,0.845359
9,LinearRegression,0.845359
7,CatBoostRegressor,0.815400
16,CatBoostRegressor,0.815400
5,RandomForestRegressor,0.792782
14,RandomForestRegressor,0.791055
17,AdaBoostRegressor,0.788300
8,AdaBoostRegressor,0.782345
